# 爬取腾讯数据

In [8]:
import pymysql
import time
import json
import traceback # 追踪异常
import requests

In [9]:
# 获取腾讯数据的封装函数
def get_tencent_data():
    url="https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
    his_url="https://view.inews.qq.com/g2/getOnsInfo?name=disease_other"
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}
    r=requests.get(url,headers)
    res = json.loads(r.text)
    data_all=json.loads(res['data'])
    
    r1=requests.get(his_url,headers)
    res1 = json.loads(r1.text)
    data_all_1=json.loads(res1['data'])
    
    history = {}
    year = 2021
    for i in data_all_1["chinaDayList"]:
        if i["date"] =="01.01":
           year = year+1
        ds =str(year)+"."+i["date"]
        tup=time.strptime(ds,"%Y.%m.%d")
        ds=time.strftime("%Y-%m-%d",tup)
        confirm=i["confirm"]
        suspect=i["suspect"]
        heal=i["heal"]
        dead=i["dead"]
        history[ds]={"confirm":confirm,"suspect":suspect,"heal":heal,"dead":dead}
      #  print(history[ds])
        
    year = 2021
    for i in data_all_1["chinaDayList"]:
        if i["date"] =="01.01":
           year = year+1
        ds =str(year)+"."+i["date"]
        tup=time.strptime(ds,"%Y.%m.%d")
        ds=time.strftime("%Y-%m-%d",tup)
        confirm=i["confirm"]
        suspect=i["suspect"]
        heal=i["heal"]
        dead=i["dead"]
        history[ds].update({"confirm":confirm,"suspect":suspect,"heal":heal,"dead":dead})
    
    details=[] # 当日详细数据
    update_time = data_all["lastUpdateTime"]
    data_country=data_all["areaTree"]  #国家列表
    data_province=data_country[0]["children"] # 中国各省
    for pro_infos in data_province:
        province = pro_infos["name"] #省名
        for city_infos in pro_infos["children"]:
            city=city_infos["name"]
            confirm=city_infos["total"]["confirm"]
            confirm_add=city_infos["today"]["confirm"]
            heal=city_infos["total"]["heal"]
            dead=city_infos["total"]["dead"]
            details.append([update_time,province,city,confirm,confirm_add,heal,dead])
    print(data_all_1.keys())
    return history,details

In [10]:
get_tencent_data()

dict_keys(['cityStatis', 'chinaDayList', 'chinaDayAddList', 'provinceCompare', 'nowConfirmStatis', 'statisGradeCityDetail', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList'])


({'2021-02-07': {'confirm': 140821,
   'suspect': 0,
   'heal': 127468,
   'dead': 5700},
  '2021-02-08': {'confirm': 141128,
   'suspect': 0,
   'heal': 127639,
   'dead': 5700},
  '2021-02-09': {'confirm': 141846,
   'suspect': 1,
   'heal': 127856,
   'dead': 5702},
  '2021-02-10': {'confirm': 142298,
   'suspect': 0,
   'heal': 128320,
   'dead': 5703},
  '2021-02-11': {'confirm': 142980,
   'suspect': 1,
   'heal': 128649,
   'dead': 5703},
  '2021-02-12': {'confirm': 143243,
   'suspect': 0,
   'heal': 128901,
   'dead': 5706},
  '2021-02-13': {'confirm': 143706,
   'suspect': 0,
   'heal': 128984,
   'dead': 5706},
  '2021-02-14': {'confirm': 145288,
   'suspect': 3,
   'heal': 129644,
   'dead': 5708},
  '2021-02-15': {'confirm': 146040,
   'suspect': 1,
   'heal': 129958,
   'dead': 5712},
  '2021-02-16': {'confirm': 146980,
   'suspect': 3,
   'heal': 130287,
   'dead': 5715},
  '2021-02-17': {'confirm': 147593,
   'suspect': 6,
   'heal': 130585,
   'dead': 5726},
  '2021-02

In [11]:
#pymysql的使用

#建立连接
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='20011026LSC0535@', db='cov', charset='utf8')
#创建游标，默认是元组型
cursor=conn.cursor()
sql="show databases"
cursor.execute(sql)
#conn.commit()#提交事务
res = cursor.fetchall()# 获得查询结果
print(cursor)
cursor.close()
conn.close()

In [12]:
def get_conn():
    """
    :return 连接 游标
    """
    #创建连接
    #建立连接
    conn = pymysql.connect(host='127.0.0.1', port=3306,user='root', password='20011026LSC0535@', db='cov', charset='utf8')
    #创建游标，默认是元组型
    cursor=conn.cursor()
    return conn,cursor

def close_conn(conn,cursor):
    conn,cursor=get_conn()
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [13]:
def update_details():
    """
    更新 details 表
    :return:
    """
    cursor = None
    conn = None
    try:
        li = get_tencent_data()[1]  #  0 是历史数据字典,1 最新详细数据列表
        conn, cursor = get_conn()
        sql = "insert into details(update_time,province,city,confirm,confirm_add,heal,dead) values(%s,%s,%s,%s,%s,%s,%s)"
        sql_query = 'select %s=(select update_time from details order by id desc limit 1)' #对比当前最大时间戳
        cursor.execute(sql_query,li[0][0])
        if not cursor.fetchone()[0]:
            print(f"{time.asctime()}开始更新最新数据")
            for item in li:
                cursor.execute(sql, item)
            conn.commit()  # 提交事务 update delete insert操作
            print(f"{time.asctime()}更新最新数据完毕")
        else:
            print(f"{time.asctime()}已是最新数据！")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [14]:
def insert_history():
    """
        插入历史数据
    :return:
    """
    cursor = None
    conn = None
    try:
        dic = get_tencent_data()[0]  # 0 是历史数据字典,1 最新详细数据列表
        print(f"{time.asctime()}开始插入历史数据")
        conn, cursor = get_conn()
        sql = "insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        for k, v in dic.items():
            # item 格式 {'2020-01-13': {'confirm': 41, 'suspect': 0, 'heal': 0, 'dead': 1}
            cursor.execute(sql, [k, v.get("confirm"), v.get("confirm_add"), v.get("suspect"),
                                 v.get("suspect_add"), v.get("heal"), v.get("heal_add"),
                                 v.get("dead"), v.get("dead_add")])

        conn.commit()  # 提交事务 update delete insert操作
        print(f"{time.asctime()}插入历史数据完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [15]:
def update_history():
    """
    更新历史数据
    :return:
    """
    cursor = None
    conn = None
    try:
        dic = get_tencent_data()[0]  #  0 是历史数据字典,1 最新详细数据列表
        print(f"{time.asctime()}开始更新历史数据")
        conn, cursor = get_conn()
        sql = "insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        sql_query = "select confirm from history where ds=%s"
        for k, v in dic.items():
            # item 格式 {'2020-01-13': {'confirm': 41, 'suspect': 0, 'heal': 0, 'dead': 1}
            if not cursor.execute(sql_query, k):
                cursor.execute(sql, [k, v.get("confirm"), v.get("confirm_add"), v.get("suspect"),
                                     v.get("suspect_add"), v.get("heal"), v.get("heal_add"),
                                     v.get("dead"), v.get("dead_add")])
        conn.commit()  # 提交事务 update delete insert操作
        print(f"{time.asctime()}历史数据更新完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [16]:
insert_history()

dict_keys(['cityStatis', 'chinaDayList', 'chinaDayAddList', 'provinceCompare', 'nowConfirmStatis', 'statisGradeCityDetail', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList'])
Sat Apr  9 16:19:56 2022开始插入历史数据
Sat Apr  9 16:19:56 2022插入历史数据完毕


In [17]:
update_details()

dict_keys(['cityStatis', 'chinaDayList', 'chinaDayAddList', 'provinceCompare', 'nowConfirmStatis', 'statisGradeCityDetail', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList'])
Sat Apr  9 16:20:02 2022开始更新最新数据
Sat Apr  9 16:20:02 2022更新最新数据完毕


## 爬取百度热搜数据

In [31]:
import selenium
from selenium.webdriver import Chrome,ChromeOptions
def get_baidu_hot():
    """
    :return: 返回百度疫情热搜
    """
    option = ChromeOptions()  # 创建谷歌浏览器实例
    option.add_argument("--headless")  # 隐藏浏览器
    option.add_argument('--no-sandbox')

    url="https://top.baidu.com/board?tab=realtime&sa=fyb_realtime_31065"
    browser = Chrome(options=option,executable_path="./chromedriver.exe")
    browser.get(url)
    # 找到展开按钮
    c=browser.find_elements_by_xpath('//*[@id="sanRoot"]/main/div[2]/div/div[2]/div/div[2]/a/div[1]')
    d=browser.find_elements_by_xpath('//*[@id="sanRoot"]/main/div[2]/div/div[2]/div/div[1]/div[2]')
    context1 = [i.text for i in c]  # 获取标签内容
    context2 = [q.text for q in d]
    a=0
    context=[]
    while a<len(context2):
        context.append(context1[a]+context2[a])
        a=a+1
    print(context)
    return context


In [32]:
def update_hotsearch():
    """
    将疫情热搜插入数据库
    :return:
    """
    cursor = None
    conn = None
    try:
        context = get_baidu_hot()
        print(f"{time.asctime()}开始更新热搜数据")
        conn, cursor = get_conn()
        sql = "insert into hotsearch(dt,content) value(%s,%s)"
        ts = time.strftime("%Y-%m-%d %X")
        for i in context:
            cursor.execute(sql,(ts,i))  # 插入数据
        conn.commit()  # 提交事务保存数据
        print(f"{time.asctime()}数据更新完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [33]:
update_hotsearch()

C:\Users\dragon\AppData\Local\Temp\ipykernel_17644\3947642194.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome(options=option,executable_path="./chromedriver.exe")
Traceback (most recent call last):
  File "C:\Users\dragon\AppData\Local\Temp\ipykernel_17644\3563112707.py", line 9, in update_hotsearch
    context = get_baidu_hot()
  File "C:\Users\dragon\AppData\Local\Temp\ipykernel_17644\3947642194.py", line 12, in get_baidu_hot
    browser = Chrome(options=option,executable_path="./chromedriver.exe")
  File "D:\Anaconda\Anaconda3\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 70, in __init__
    super(WebDriver, self).__init__(DesiredCapabilities.CHROME['browserName'], "goog",
  File "D:\Anaconda\Anaconda3\lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 93, in __init__
    RemoteWebDriver.__init__(
  File "D:\Anaconda\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py",